In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/08/07 17:05:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2022-08-07 17:05:10--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.225.141.174, 13.225.141.105, 13.225.141.158, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.225.141.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet’

fhvhv_tripdata_2021 100%[===================>] 294.61M  58.6MB/s    in 5.0s    

2022-08-07 17:05:15 (59.5 MB/s) - ‘fhvhv_tripdata_2021-01.parquet’ saved [308924937/308924937]



In [4]:
!wc -l fhvhv_tripdata_2021-01.parquet

1006794 fhvhv_tripdata_2021-01.parquet


In [5]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [6]:
df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [7]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(originating_base_num,StringType,true),StructField(request_datetime,TimestampType,true),StructField(on_scene_datetime,TimestampType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropoff_datetime,TimestampType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(trip_miles,DoubleType,true),StructField(trip_time,LongType,true),StructField(base_passenger_fare,DoubleType,true),StructField(tolls,DoubleType,true),StructField(bcf,DoubleType,true),StructField(sales_tax,DoubleType,true),StructField(congestion_surcharge,DoubleType,true),StructField(airport_fee,DoubleType,true),StructField(tips,DoubleType,true),StructField(driver_pay,DoubleType,true),StructField(shared_request_flag,StringType,true),StructField(shared_match_flag,StringType,true),StructField(access_a_ride_flag,StringType,true),StructField(wav_req

In [8]:
!head -n 1001 fhvhv_tripdata_2021-01.parquet > head.parquet

In [9]:
import pandas as pd

In [10]:
df_pandas = pd.read_parquet('fhvhv_tripdata_2021-01.parquet',engine='pyarrow')

In [11]:
df_pandas.dtypes

hvfhs_license_num               object
dispatching_base_num            object
originating_base_num            object
request_datetime        datetime64[ns]
on_scene_datetime       datetime64[ns]
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int64
DOLocationID                     int64
trip_miles                     float64
trip_time                        int64
base_passenger_fare            float64
tolls                          float64
bcf                            float64
sales_tax                      float64
congestion_surcharge           float64
airport_fee                    float64
tips                           float64
driver_pay                     float64
shared_request_flag             object
shared_match_flag               object
access_a_ride_flag              object
wav_request_flag                object
wav_match_flag                  object
dtype: object

In [12]:
# spark.createDataFrame(df_pandas).show()

Integer - 4 bytes
Long - 8 bytes

In [28]:
from pyspark.sql import types

In [14]:
# schema = types.StructType([
#     types.StructField('hvfhs_license_num', types.StringType(), True),
#     types.StructField('dispatching_base_num', types.StringType(), True),
#     types.StructField('pickup_datetime', types.TimestampType(), True),
#     types.StructField('dropoff_datetime', types.TimestampType(), True),
#     types.StructField('PULocationID', types.IntegerType(), True),
#     types.StructField('DOLocationID', types.IntegerType(), True),
#     types.StructField('SR_Flag', types.StringType(), True)
# ])

In [15]:
# df = spark.read \
#     .option("header", "true") \
#     .schema(schema) \
#     .csv('fhvhv_tripdata_2021-01.csv')

In [16]:
df = df.repartition(24)

In [17]:
df.write.parquet('fhvhv/2021/01/', mode = 'overwrite')

In [18]:
df = spark.read.parquet('fhvhv/2021/01/')

In [19]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [21]:
df.select('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003') \
    .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-15 14:11:07|2021-01-15 14:41:51|         142|         107|
|2021-01-29 18:20:30|2021-01-29 18:32:03|         159|          78|
|2021-01-11 21:49:41|2021-01-11 22:00:20|         158|          87|
|2021-01-09 07:49:14|2021-01-09 08:02:16|         118|           6|
|2021-01-27 17:56:55|2021-01-27 18:11:04|         116|          50|
|2021-01-23 15:15:15|2021-01-23 15:25:35|          17|         189|
|2021-01-29 16:05:00|2021-01-29 16:20:57|         205|         215|
|2021-01-30 23:46:53|2021-01-31 00:00:42|          69|         212|
|2021-01-28 21:56:06|2021-01-28 21:59:55|          20|          78|
|2021-01-07 20:32:43|2021-01-07 20:53:50|          82|          75|
|2021-01-26 20:35:42|2021-01-26 20:45:22|          47|          59|
|2021-01-21 14:07:53|2021-01-21 14:21:52|       

In [22]:
from pyspark.sql import functions as F

In [23]:
df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [25]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [26]:
crazy_stuff('B02884')

's/b44'

In [29]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [31]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date','pickup_datetime', 'dropoff_datetime',\
            'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+-------------------+-------------------+------------+------------+
|base_id|pickup_date|dropoff_date|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------+-----------+------------+-------------------+-------------------+------------+------------+
|  e/9ce| 2021-01-14|  2021-01-14|2021-01-14 06:10:27|2021-01-14 06:16:48|          17|          61|
|  e/9ce| 2021-01-14|  2021-01-14|2021-01-14 06:53:43|2021-01-14 07:06:18|          61|          71|
|  e/acc| 2021-01-15|  2021-01-15|2021-01-15 14:11:07|2021-01-15 14:41:51|         142|         107|
|  a/b49| 2021-01-29|  2021-01-29|2021-01-29 18:20:30|2021-01-29 18:32:03|         159|          78|
|  e/9ce| 2021-01-07|  2021-01-07|2021-01-07 09:12:19|2021-01-07 09:18:37|         119|         235|
|  e/acc| 2021-01-11|  2021-01-11|2021-01-11 21:49:41|2021-01-11 22:00:20|         158|          87|
|  e/b32| 2021-01-09|  2021-01-09|2021-01-09 07:49:14|2021-01-09 08:02:16|         118|    

In [33]:
!head -n 10 head.parquet

PAR1<HL  �      
cc``�3000f��L�, �)Fs    ��S��/,���6 (HV0005HV0003   �      
�o{��=�0V>`�!���C���`
�R��N��.��>�,��ϼ;��n�-���$6�]X�y�~�o��O���
;)�a��}֐�9�������)��e݄4?	�\�h+b/��6������B
�}Ա]]j��A��|v�m��l_7�c�:�Z��(�Ű�5����O��8c�R�Å�3d{�a؍%|�{��ޮ��_��Ga�.�)�`�bKl����k���v;k��S_�k���d��<����o�ϳ��{O��}�~?${ǜ{��9��.����4v�u
;S��Sن|l��V.�_���h������&��}��mȇv��0q�Cmg{^V"�c�Kc��́�m@���J?���o��6~�}�<�k;����I��d�f�E�v��"�Pg�6��	��ƣ�� �c�I�?�6�5ڦ�S����p߮eNM;��t۝p
k�ؚg}5O�>����q4�%��9�.<�\�}��qS�ٷ4�R��t=��vq(vڮ���~�00fg&.��+��^4co��r�C0���Ͱ�v&w�g�����<Q��T��f��G��H�ln�޸�}0�qʳ]d&����6�<S�؃�ھ�����6c1Oc�h�ui͚����;0W��-l	�:�Cmp\�9<Gie����?Ѷ�7����F����)p���م9|����-���[��C{.rc����9�ض�T{�b��'YxaF�<�}{"-�k�o�1�:�!]*��T��i��,_���>����v�������6�`g��%�=�������]ɝ,�o�7����z�;�Nx��{�ɋ�.������������ߏ��8�S
w᎟��1�ɂA[���xM���r��/-	D�<��cP;g=�L��+�`��D mW@��_�6��DEԶ�Aq��M���2W1����C�#.�̛�7��D#f